In [37]:
import yfinance as yf
import matplotlib as plt
import indicators 
import os
import importlib
print(dir(indicators))

['BollingerBands', 'EMA', 'MA', 'MACD', 'RSI', 'STD', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'bollinger_bands', 'pandas', 'pd']


In [38]:
sp500 = yf.download("^GSPC", period="max", interval="1d")

sp500["V_t"] = (sp500["Close"].diff() / sp500["Close"].shift()) * 100
sp500["MA5"] = MA(sp500["Close"], 5)
sp500["MA15"] = MA(sp500["Close"], 15)
sp500["EMA5"] = EMA(sp500["Close"], 5)
sp500["EMA15"] = EMA(sp500["Close"], 15)
sp500["RSI14"] = RSI(sp500["Close"], 14)
sp500["MACD"], sp500["Signal"] = MACD(sp500["Close"], 12, 26)
sp500["STD10"] = STD(sp500["Close"], 10)
sp500["STD20"] = STD(sp500["Close"], 20)
sp500["BollingerBandUpper"], sp500["BollingerBandLower"] = bollinger_bands(sp500["Close"], 20, 2)

[*********************100%***********************]  1 of 1 completed
